Imports and global variable setups

In [14]:
from keras.models import Model
from keras.layers import Input, LSTM, Dense
import numpy as np

batch_size = 1000
epochs = 20
latent_dim = 256 # what is this?
num_samples = 500000
data_path = "data/names_initials.csv"

Read in and vectorize the data

In [15]:
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()
with open(data_path, "r", encoding="utf-8") as f:
    lines = f.read().split("\n")
for line in lines[ : min(num_samples, len(lines) - 1)]:
    input_text, target_text = line.split(",")
    #Now, use "\t" as the start sequence character and "\n" as the end sequence character
    target_text = "\t" + target_text + "\n"
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(seq) for seq in input_texts])
max_decoder_seq_length = max([len(seq) for seq in target_texts])

print("Number of samples:", len(input_texts))
print("Number of unique input tokens:", num_encoder_tokens)
print("Number of unique output tokens", num_decoder_tokens)
print("Max sequence length for inputs:", max_encoder_seq_length)
print("Max sequence length for outputs:", max_decoder_seq_length)

input_token_index = dict([(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict([(char, i) for i, char in enumerate(target_characters)])

encoder_input_data = np.zeros((len(input_texts), max_encoder_seq_length, num_encoder_tokens),
                              dtype="float32")
decoder_input_data = np.zeros((len(target_texts), max_decoder_seq_length, num_decoder_tokens),
                              dtype="float32")
decoder_target_data = np.zeros((len(target_texts), max_decoder_seq_length, num_decoder_tokens),
                              dtype="float32")

for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one time step
        decoder_input_data[i, t, target_token_index[char]] = 1
        if t > 0:
            #decoder_target_data is ahead by one time step and doesn't include the start char
            decoder_target_data[i, t-1, target_token_index[char]] = 1

print(encoder_input_data.shape)
print(decoder_input_data.shape)
print(decoder_target_data.shape)


Number of samples: 500000
Number of unique input tokens: 53
Number of unique output tokens 28
Max sequence length for inputs: 23
Max sequence length for outputs: 4
(500000, 23, 53)
(500000, 4, 28)
(500000, 4, 28)


Define an input sequence and process it

In [16]:
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# Discard encoder_outputs and only keep the states
encoder_states = [state_h, state_c]

# Set up the decoder and feed the encoder_states as its initial states
decoder_inputs = Input(shape=(None, num_decoder_tokens))
# We set up our decoder to return full output sequences, and to return internal states as well.
# We don't use the return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation="softmax")
decoder_outputs = decoder_dense(decoder_outputs)

Define the model that will turn `encoder_input_data` and `decoder_input_data` into `decoder_target_data`

In [17]:
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

Train and save the model

In [18]:
model.compile(optimizer="rmsprop", loss="categorical_crossentropy", metrics=["acc"])
model.fit([encoder_input_data, decoder_input_data], decoder_target_data, batch_size=batch_size,
          epochs=epochs, validation_split=0.2)
model.save("seq2seq-500000.h5") # save model

Train on 400000 samples, validate on 100000 samples
Epoch 1/20
400000/400000 [==============================] - 693s 2ms/step - loss: 1.2563 - acc: 0.3624 - val_loss: 0.9114 - val_acc: 0.4646
Epoch 2/20
400000/400000 [==============================] - 570s 1ms/step - loss: 0.7772 - acc: 0.5099 - val_loss: 0.6991 - val_acc: 0.5353
Epoch 3/20
400000/400000 [==============================] - 637s 2ms/step - loss: 0.6329 - acc: 0.5587 - val_loss: 0.4811 - val_acc: 0.6093
Epoch 4/20
400000/400000 [==============================] - 630s 2ms/step - loss: 0.3003 - acc: 0.6625 - val_loss: 0.0951 - val_acc: 0.7208
Epoch 5/20
400000/400000 [==============================] - 476s 1ms/step - loss: 0.0353 - acc: 0.7409 - val_loss: 0.0022 - val_acc: 0.7497
Epoch 6/20
400000/400000 [==============================] - 472s 1ms/step - loss: 0.0090 - acc: 0.7478 - val_loss: 2.6373e-04 - val_acc: 0.7500
Epoch 7/20
400000/400000 [==============================] - 474s 1ms/step - loss: 0.0057 - acc: 0.7488 -

KeyboardInterrupt: 